In [3]:
import os
import sys
import pandas as pd
import win32com.client

# CONFIG
folder_path = r"C:\Users\Rutvik Desai\Monitoring Control & Compliance\Technician - Documents\Projects\Green For Life (GFL)\Arbor Hills Landfill\Raw Data\2025\08-25\CSV"
appended_file_path = r"C:\Users\Rutvik Desai\Monitoring Control & Compliance\Technician - Documents\Projects\Green For Life (GFL)\Arbor Hills Landfill\Raw Data\2025\AHL_Rolling_python.xlsx"

rolling_columns = [
    'Well_ID', 'WellHead_Type', 'Date', 'CH4_Percent', 'CO2_Percent', 'O2_Percent', 'Balance_Percent',
    'Init_Temperature_F', 'Final_Temperature_F', 'Init_Static_P', 'Adj_Static_P', 'H2_Percent', 'CO_PPM',
    'H2S_PPM', 'Press. HOV', 'Temp. HOV', 'Init_Diff_P', 'Adj_Diff_P', 'Init_Flow', 'Adj_Flow',
    'Init_Avail_P', 'Adj_Avail_P', 'Valve_Adjustment', 'Valve_Comment', 'Well_Comment', 'Pump Counts', 'Technician'
]

def prompt_close_excel():
    print("⚠️ Warning: This script will close ALL open Excel windows WITHOUT saving any unsaved work!")
    answer = input("Do you want to continue and close Excel? (yes/no): ").strip().lower()
    if answer not in ('yes', 'y'):
        print("Operation cancelled by user.")
        sys.exit(0)

def close_excel():
    try:
        excel = win32com.client.Dispatch("Excel.Application")
        excel.Application.Quit()
        print("✅ Excel closed successfully.")
    except Exception as e:
        print(f"⚠️ Could not close Excel or Excel not running: {e}")

def main():
    prompt_close_excel()
    close_excel()

    if not os.path.exists(appended_file_path):
        print(f"❌ Excel file not found: {appended_file_path}")
        sys.exit(1)

    try:
        with pd.ExcelFile(appended_file_path) as xl:
            appended_df = xl.parse('Appended_Data') if 'Appended_Data' in xl.sheet_names else pd.DataFrame()
            rolling_df = xl.parse('rolling_data') if 'rolling_data' in xl.sheet_names else pd.DataFrame()
            base_columns = [col for col in appended_df.columns if col != 'Source_File']
    except Exception as e:
        print(f"❌ Error reading Excel file: {e}")
        sys.exit(1)

    # Free resource
    del xl

    already_appended = set(appended_df['Source_File']) if 'Source_File' in appended_df.columns else set()
    processed_files = []

    for filename in os.listdir(folder_path):
        # Skip files containing 'probe' in the filename (case-insensitive)
        if "probe" in filename.lower():
            continue

        if filename.endswith(".csv") and filename not in already_appended:
            file_path = os.path.join(folder_path, filename)
            try:
                new_df = pd.read_csv(file_path)

                # Replace NR and NA (case insensitive, with spaces) with None/blank
                new_df.replace(to_replace=r'^\s*(NR|NA)\s*$', value=None, regex=True, inplace=True)

                # Replace '>>>' in H2S_PPM (with optional surrounding spaces) with 2001
                if 'H2S_PPM' in new_df.columns:
                    new_df['H2S_PPM'] = new_df['H2S_PPM'].replace(r'^\s*>>>\s*$', 2001, regex=True)

                # Filter and reorder columns to match appended_df base columns
                filtered_cols = [col for col in base_columns if col in new_df.columns]
                filtered_df = new_df[filtered_cols].copy()

                # Add any missing columns from base_columns as blank
                for col in base_columns:
                    if col not in filtered_df.columns:
                        filtered_df[col] = None

                filtered_df = filtered_df[base_columns]

                # Add Source_File column
                filtered_df['Source_File'] = filename

                # Append a blank row after the data
                blank_row = pd.DataFrame({col: [None] for col in filtered_df.columns})

                # Append to main appended_df
                appended_df = pd.concat([appended_df, filtered_df, blank_row], ignore_index=True)

                # Prepare rolling_data subset (only the rolling_columns present in filtered_df)
                matching_cols = [col for col in rolling_columns if col in filtered_df.columns]
                rolling_filtered = filtered_df[matching_cols]

                # Append rolling_filtered to rolling_df
                rolling_df = pd.concat([rolling_df, rolling_filtered], ignore_index=True)

                processed_files.append(filename)
            except Exception as e:
                print(f"⚠️ Error processing file '{filename}': {e}")

    try:
        with pd.ExcelWriter(appended_file_path, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
            appended_df.to_excel(writer, sheet_name='Appended_Data', index=False)
            rolling_df.to_excel(writer, sheet_name='rolling_data', index=False)
    except PermissionError:
        print("❌ Unable to save. Please close the Excel file and try again.")
        sys.exit(1)

    if processed_files:
        print("\n✅ Processed CSV files:")
        for f in processed_files:
            print(f"  - {f}")
        print("\n📄 'Appended_Data' and 'rolling_data' sheets updated.")
    else:
        print("\nℹ️ No new CSV files to process.")

if __name__ == "__main__":
    main()


⚠️ Warning: This script will close ALL open Excel windows WITHOUT saving any unsaved work!


Do you want to continue and close Excel? (yes/no):  yes


✅ Excel closed successfully.


C:\Users\Rutvik Desai\AppData\Local\Temp\ipykernel_31788\1862573909.py:90: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  appended_df = pd.concat([appended_df, filtered_df, blank_row], ignore_index=True)
C:\Users\Rutvik Desai\AppData\Local\Temp\ipykernel_31788\1862573909.py:97: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  rolling_df = pd.concat([rolling_df, rolling_filtered], ignore_index=True)
C:\Users\Rutvik Desai\AppData\Local\Temp\ipykernel_31788\1862573909.py:90: FutureWarning: The behavior


✅ Processed CSV files:
  - AHL080825MC.csv
  - AHL080825PP.csv
  - AHL081125AS.csv
  - AHL081125HCHPP.csv
  - AHL081125MC.csv
  - AHL081225AS.csv
  - AHL081225HCHPP.csv
  - AHL081225MC.csv
  - AHL081225NG.csv

📄 'Appended_Data' and 'rolling_data' sheets updated.
